In [1]:
import torch
from llama_cpp import Llama

In [6]:
import gguf
import numpy as np

def load_embedding_weights_from_gguf(model_path):
    """
    Carga los pesos del embedding directamente desde un archivo GGUF.

    Args:
        model_path (str): Ruta al archivo .gguf del modelo.

    Returns:
        numpy.ndarray: Un array 2D con los pesos del embedding (vocab_size, embedding_dim).
                       Retorna None si no se encuentra el tensor.
    """
    try:
        # 1. Cargar el archivo GGUF
        reader = gguf.GGUFReader(model_path, 'r')

        # 2. Iterar por los tensores para encontrar el de embedding
        # El nombre puede variar, pero suele ser 'token_embd.weight'
        embedding_tensor_name = 'token_embd.weight' # Nombre común
        found_tensor = None

        for tensor in reader.tensors:
            if tensor.name == embedding_tensor_name:
                print(f"Tensor encontrado: {tensor.name}, Shape: {tensor.shape}, Tipo: {tensor.tensor_type}")
                found_tensor = tensor
                break
        else:
            raise ValueError(f"No se encontró el tensor de embedding con el nombre '{embedding_tensor_name}' en {model_path}")

        # 3. Extraer los datos
        # gguf.Reader nos da los datos ya deserializados y descuantizados (si es necesario)
        # como un numpy array. El acceso es mediante found_tensor.data
        print(found_tensor)
        embedding_weights = found_tensor.data
        print(embedding_weights[0])

        # Asegurarse de que es un array 2D
        # La forma típica es (n_vocab, n_embd) en el archivo,
        # aunque internamente ggml/gguf puede almacenarlo transpuesto.
        # gguf.Reader debería manejar esto y devolver la forma correcta.
        # Verifica la forma:
        print(f"Pesos del embedding cargados. Shape: {embedding_weights.shape}, Dtype: {embedding_weights.dtype}")

        return embedding_weights

    except FileNotFoundError:
        print(f"Error: No se pudo encontrar el archivo {model_path}")
    except Exception as e:
        print(f"Error al cargar el archivo GGUF: {e}")
        import traceback
        traceback.print_exc()
    return None

In [7]:
# --- Ejemplo de uso ---
model_path = "/home/giorgio6846/Code/Sign-AI/local_models/Llama-3.2-3B-Instruct-GGUF/Llama-3.2-3B-Instruct-Q4_K_M.gguf"
embedding_weights = load_embedding_weights_from_gguf(model_path)
#
if embedding_weights is not None:
    print(f"Shape final: {embedding_weights.shape}")
else:
    print("No se pudieron cargar los pesos del embedding.")


Tensor encontrado: token_embd.weight, Shape: [  3072 128256], Tipo: 14
ReaderTensor(name='token_embd.weight', tensor_type=<GGMLQuantizationType.Q6_K: 14>, shape=memmap([  3072, 128256], dtype=uint64), n_elements=394002432, n_bytes=323205120, data_offset=7850432, data=memmap([[129,  67,  45, ...,  77, 204,   0],
        [253,  96, 224, ..., 181,   4,   1],
        [170, 154, 208, ..., 100, 228, 128],
        ...,
        [ 20,  45, 233, ..., 238,  45, 130],
        [ 20,  45, 233, ..., 238,  45, 130],
        [ 20,  45, 233, ..., 238,  45, 130]],
       shape=(128256, 2520), dtype=uint8), field=ReaderField(offset=7822856, name='token_embd.weight', parts=[memmap([17], dtype=uint64), memmap([116, 111, 107, 101, 110,  95, 101, 109,  98, 100,  46, 119, 101,
        105, 103, 104, 116], dtype=uint8), memmap([2], dtype=uint32), memmap([  3072, 128256], dtype=uint64), memmap([14], dtype=uint32), memmap([12544], dtype=uint64)], data=[1, 3, 4, 5], types=[]))
[129  67  45 ...  77 204   0]
Pesos d

In [4]:
path_model = "/home/giorgio6846/Code/Sign-AI/local_models/Llama-3.2-3B-Instruct-GGUF/Llama-3.2-3B-Instruct-Q4_K_M.gguf"
model_cpp = Llama(model_path=path_model, n_ctx=1024, embedding=True, logits_all=True, verbose=False,) #n_gpu_layers=-1)

llama_context: n_ctx_per_seq (1024) < n_ctx_train (131072) -- the full capacity of the model will not be utilized
llama_kv_cache_unified: LLAMA_SET_ROWS=0, using old ggml_cpy() method for backwards compatibility


In [9]:
target = "un texto normal como cualquiera"
embeddings_cpp = torch.tensor(model_cpp.embed(target))
embeddings_cpp.shape

torch.Size([8, 3072])